In [4]:
from deepfindET.utils import copick_tools
import copick
import os
import numpy as np

In [5]:
copick_config_path = "working/copick.json"
copick_root = copick.from_file(copick_config_path)

runs = os.listdir('Extracted_Data/train/static/ExperimentRuns')
voxel_size = 10
tomogram_algo = 'denoised'

out_name = 'remotetargets'
out_user_id = 'deepfindET'
out_session_id = '0'

In [6]:
from sklearn.model_selection import train_test_split
from keras._tf_keras.keras.utils import to_categorical
import keras._tf_keras.keras as keras
from keras._tf_keras.keras.losses import dice, CategoricalFocalCrossentropy
from model  import My_LATUP
from patchify import patchify
import tensorflow as tf

class total_loss(keras.losses.Loss):
    def call(self, y_true, y_pred):
        dice_loss = dice(y_true, y_pred)
        focal_loss = CategoricalFocalCrossentropy(from_logits=True)(y_true, y_pred)
        return dice_loss +(1* focal_loss)


model = My_LATUP(input_shape = (64, 64, 64, 3), loss=dice)
model.summary(line_length=110)

Model: "MY_LATUP"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                   ┃ Output Shape              ┃          Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)     │ (None, 64, 64, 64, 3)     │                0 │ -                          │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_PC_Embedded_Layer (Conv3D)  │ (None, 64, 64, 64, 32)    │            2,624 │ input_layer_1[0][0]        │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_PC_Conv1_Layer (Conv3D)     │ (None, 64, 64, 64, 32)    │            1,056 │ E1_PC_Embedded_Layer[0][0] │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_PC_Conv2_Layer (Conv3D)     │ (None, 64, 64, 64, 32)    │           27,680 │ E1_PC_Embedded_Layer[0][0] │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_PC_Conv3_Layer (Conv3D)     │ (None, 64, 64, 64, 32)    │          128,032 │ E1_PC_Embedded_Layer[0][0] │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_maxpool1_Layer              │ (None, 32, 32, 32, 32)    │                0 │ E1_PC_Conv1_Layer[0][0]    │
│ (MaxPooling3D)                 │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_maxpool2_Layer              │ (None, 32, 32, 32, 32)    │                0 │ E1_PC_Conv2_Layer[0][0]    │
│ (MaxPooling3D)                 │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_maxpool3_Layer              │ (None, 32, 32, 32, 32)    │                0 │ E1_PC_Conv3_Layer[0][0]    │
│ (MaxPooling3D)                 │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_concat_Layer (Concatenate)  │ (None, 32, 32, 32, 96)    │                0 │ E1_maxpool1_Layer[0][0],   │
│                                │                           │                  │ E1_maxpool2_Layer[0][0],   │
│                                │                           │                  │ E1_maxpool3_Layer[0][0]    │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E2_SE1_Layer (SE_Layer)        │ (None, 32, 32, 32, 96)    │            2,412 │ E1_concat_Layer[0][0]      │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E2_Conv1_Layer (Conv3D)        │ (None, 32, 32, 32, 64)    │          165,952 │ E2_SE1_Layer[0][0]         │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E2_instance_Layer              │ (None, 32, 32, 32, 64)    │              128 │ E2_Conv1_Layer[0][0]       │
│ (InstanceNormalization)        │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E2_Conv2_Layer (Conv3D)        │ (None, 32, 32, 32, 64)    │          110,656 │ E2_instance_Layer[0][0]    │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E2_Drop (Dropout)              │ (None, 32, 32, 32, 64)    │                0 │ E2_Conv2_Layer[0][0]       │
├───

 Total params: 3,575,591 (13.64 MB)

 Trainable params: 3,575,591 (13.64 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
def get_tomogram(run_id):
    return copick_tools.get_copick_tomogram(copick_root,
                                                voxelSize = voxel_size,
                                                tomoAlgorithm=tomogram_algo,
                                                tomoID=run_id)

def get_mask(run_id):
    return copick_tools.get_copick_segmentation(copick_root.get_run(run_id),
                                                segmentationName=out_name,
                                                userID=out_user_id,
                                                sessionID=out_session_id)

def shape_t(obj):
    obj = patchify(obj, (64, 64, 64), step=64)
    obj = np.reshape(obj, (-1, obj.shape[3], obj.shape[4], obj.shape[5]))
    return obj


def train(run_ids):
    for run_id in run_ids:
        tomogram = np.array(get_tomogram(run_id))
        mask = np.array(get_mask(run_id))

        tomogram = shape_t(tomogram)
        mask = shape_t(mask)
        tomogram = np.stack((tomogram,)*3, axis=-1)

        mask = np.expand_dims(mask, axis=4)
        mask = to_categorical(mask, num_classes=7)

        x_train, x_test, y_train, y_test = train_test_split(tomogram, mask, test_size=0.1, random_state=42)
        model.fit(x_train, y_train,epochs=20, batch_size=4, validation_data=(x_test, y_test))

        print(f'Training done for Run_id:{run_id}')


In [8]:
train(runs)

Epoch 1/20


I0000 00:00:1736182660.808066  418240 service.cc:148] XLA service 0x7f4af0014630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736182660.808700  418240 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti Laptop GPU, Compute Capability 8.6
2025-01-06 22:27:40.999021: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1736182661.186613  418240 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_non_negative/assert_less_equal/Assert/AssertGuard/Assert
W0000 00:00:1736182661.314033  418240 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_non_negative_1/assert_less_equal/Assert/AssertGuard/Assert
W0000 00:00:1736182661.315284  418240 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_less/Assert/AssertGuard/Assert
W0000 00:00:1736182661.316626  418240 asse

36/37 ━━━━━━━━━━━━━━━━━━━━ 0s 570ms/step - IoU: 0.3400 - loss: 19.4231

W0000 00:00:1736182720.817193  418242 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_non_negative/assert_less_equal/Assert/AssertGuard/Assert
W0000 00:00:1736182720.900600  418242 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_non_negative_1/assert_less_equal/Assert/AssertGuard/Assert
W0000 00:00:1736182720.901349  418242 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_less/Assert/AssertGuard/Assert
W0000 00:00:1736182720.902097  418242 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_less_1/Assert/AssertGuard/Assert


37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - IoU: 0.3386 - loss: 19.3761   

W0000 00:00:1736182738.201772  418242 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_non_negative/assert_less_equal/Assert/AssertGuard/Assert
W0000 00:00:1736182738.271628  418242 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_non_negative_1/assert_less_equal/Assert/AssertGuard/Assert
W0000 00:00:1736182738.273129  418242 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_less/Assert/AssertGuard/Assert
W0000 00:00:1736182738.274614  418242 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_less_1/Assert/AssertGuard/Assert
W0000 00:00:1736182740.702326  418241 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_non_negative/assert_less_equal/Assert/AssertGuard/Assert
W0000 00:00:1736182740.751428  418241 assert_op.cc:38] Ignoring Assert operator confusion_matrix/assert_non_negative_1/assert_less_equal/Assert/AssertGuard/Assert
W0000 00:00:1736182740.752085  418241 assert_op.cc:38] Ignoring Assert operator 

37/37 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - IoU: 0.3372 - loss: 19.3317 - val_IoU: 0.2500 - val_loss: 14.2737
Epoch 2/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 541ms/step - IoU: 0.2513 - loss: 13.0716 - val_IoU: 0.2636 - val_loss: 9.8495
Epoch 3/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 548ms/step - IoU: 0.2689 - loss: 8.9964 - val_IoU: 0.2864 - val_loss: 6.7221
Epoch 4/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 555ms/step - IoU: 0.2952 - loss: 6.1264 - val_IoU: 0.3200 - val_loss: 4.5408
Epoch 5/20


2025-01-06 22:30:02.762924: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 41943040 bytes after encountering the first element of size 41943040 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 540ms/step - IoU: 0.3319 - loss: 4.1262 - val_IoU: 0.3655 - val_loss: 3.0381
Epoch 6/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 564ms/step - IoU: 0.3804 - loss: 2.7540 - val_IoU: 0.4185 - val_loss: 2.0150
Epoch 7/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 547ms/step - IoU: 0.4336 - loss: 1.8217 - val_IoU: 0.4688 - val_loss: 1.3266
Epoch 8/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 552ms/step - IoU: 0.4808 - loss: 1.2047 - val_IoU: 0.4939 - val_loss: 0.8685
Epoch 9/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 532ms/step - IoU: 0.4934 - loss: 0.7904 - val_IoU: 0.4920 - val_loss: 0.5669
Epoch 10/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 522ms/step - IoU: 0.4915 - loss: 0.5172 - val_IoU: 0.4905 - val_loss: 0.3702
Epoch 11/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 534ms/step - IoU: 0.4901 - loss: 0.3364 - val_IoU: 0.4896 - val_loss: 0.2432
Epoch 12/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 541ms/step - IoU: 0.4895 - loss: 0.2251 - val_IoU: 0.4894 - val_loss: 0.1618
Epoch 13/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 522ms/step - 

KeyboardInterrupt: 